<a href="https://colab.research.google.com/github/ramrajv/NeuralNetworks/blob/main/House_Price_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Part 1: Import the Housing data and do feature transformations

In [2]:
url = 'https://raw.githubusercontent.com/ramrajv/NeuralNetworks/main/house_price_full.csv'
df= pd.read_csv(url)
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [3]:
X = df.copy()
# Remove target
Y = X.pop('price')

# perform a scaler transform of the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# perform log transformation of target variable (For Sandeep: Is this needed?)
Y = np.log(Y)

In [4]:
df_scaled = pd.DataFrame(X)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [5]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

## Part 2: Create Model Using `keras`

![](multiple_neurons.png)

In [6]:
from tensorflow import keras

In [15]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(X.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2)                 6         
                                                                 
 dense_5 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [16]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1) 
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [17]:
model.fit(X,Y.values,epochs=10,batch_size=32)

Epoch 1/10
16/16 [==============================] - 0s 2ms/step - loss: 106.9720
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 28.7099
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 6.3215
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 1.5509
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 0.6424
Epoch 6/10
16/16 [==============================] - 0s 2ms/step - loss: 0.4617
Epoch 7/10
16/16 [==============================] - 0s 3ms/step - loss: 0.4053
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 0.3737
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 0.3513
Epoch 10/10
16/16 [==============================] - 0s 2ms/step - loss: 0.3318


In [18]:
model.predict(X)[:,0]

array([13.078255 , 13.444735 , 12.982122 , 12.966969 , 13.323492 ,
       12.727972 , 12.617831 , 13.249454 , 12.858988 , 13.322599 ,
       13.024776 , 13.212172 , 12.8862   , 13.100668 , 13.499236 ,
       13.092197 , 13.047789 , 13.172383 , 13.074809 , 13.093902 ,
       12.909664 , 13.319717 , 13.017601 , 13.246212 , 13.324951 ,
       12.946422 , 12.878159 , 13.309579 , 13.50041  , 12.810242 ,
       13.323902 , 13.223553 , 13.326148 , 13.230818 , 13.036623 ,
       12.607399 , 12.743956 , 12.998524 , 12.9757185, 12.87001  ,
       13.046255 , 13.321047 , 12.734073 , 13.013914 , 12.653212 ,
       13.095531 , 12.973553 , 12.982122 , 13.085667 , 13.124592 ,
       13.249454 , 12.714853 , 12.77724  , 13.008263 , 13.479303 ,
       13.324099 , 13.310322 , 13.443432 , 13.0804615, 12.542199 ,
       13.124592 , 12.786343 , 13.246212 , 12.82778  , 13.212172 ,
       12.723837 , 13.035044 , 13.105444 , 12.241455 , 12.919762 ,
       13.2756405, 12.733851 , 13.303029 , 12.988389 , 13.0793